Copyright 2024 Patrick Loeber

**DISCLAIMER**:

**This guide is purely for educational purposes. The model does not replace real therapy.**

**The prompts and dataset may contain sensitive information related to mental health.**

**If you need emotional support or counseling, you'll find free helplines at [https://findahelpline.com/](https://findahelpline.com/).**


# Build a mental health companion with Mistral

In this guide, you'll learn how to build a mental health companion with Mistral.

The model learns to provide helpful advice and support based on training data that consists of counseling sessions with qualified psychologists.

We follow this approach:

- **Dataset**: We finetune Mistral 7B on a mental health counseling conversation dataset.

- **Fine-tuning**: For finetuning, we'll use the Mistral Finetuning API (This tutorial can be followed by being on the free tier).

- **Safeguarding**: We'll also employ a second LLM as "supervisor", so you can rate the severity of the user problem and can take extra care and further action if needed.





## Install dependencies

- [https://github.com/mistralai/client-python](https://github.com/mistralai/client-python)
- [https://github.com/huggingface/datasets](https://github.com/huggingface/datasets)

In [ ]:
!pip install mistralai datasets

## Prepare the dataset

We'll use the [Amod/mental_health_counseling_conversations](https://huggingface.co/datasets/Amod/mental_health_counseling_conversations) dataset.

The dataset consists of over 3500 questions and answers sourced from two online counseling and therapy platforms. The questions cover a wide range of mental health topics, and answers are provided by qualified psychologists. The dataset may contain sensitive information related to mental health. All data was anonymized and no personally identifiable information is included.

We load the data and split it into training and validation, and save the data into the required jsonl format for fine-tuning.

In [ ]:
from datasets import load_dataset

ds = load_dataset('Amod/mental_health_counseling_conversations')

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [4]:
ds_splitted = ds['train'].train_test_split(test_size=0.1)
ds_splitted

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3160
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 352
    })
})

In [83]:
# Create and save jsonl files in the correct format
import json

def create_jsonl(dataset, output_filename):
  lines = []
  for sample in dataset:
    context, response = sample["Context"], sample["Response"]

    # replace unicode characters, e.g. \u2019
    context = context.replace("\u2019", "'").replace("\u00a0", "").replace("\u201c", "'").replace("\u201d", "'")
    response = response.replace("\u2019", "'").replace("\u00a0", "").replace("\u201c", "'").replace("\u201d", "'")

    # remove remaining unicode characters
    response = response.encode("ascii", "ignore").decode('unicode-escape')
    context = context.encode("ascii", "ignore").decode('unicode-escape')

    # skip empty entries
    if not response or not context:
      continue

    # apply the format needed for the finetuning API
    line_dict = {"messages": [{"role": "user", "content": context}, {"role": "assistant", "content": response}]}
    lines.append(line_dict)

  with open(output_filename, "w") as f:
    for line in lines:
      f.write(f"{json.dumps(line)}\n")

In [12]:
create_jsonl(ds_splitted["train"], "mental_health_counseling_train.jsonl")
create_jsonl(ds_splitted["test"], "mental_health_counseling_eval.jsonl")

## Upload dataset

In [14]:
from mistralai import Mistral

api_key = "YOUR_API_KEY"

client = Mistral(api_key=api_key)

In [121]:
mental_health_counseling_train = client.files.upload(file={
    "file_name": "mental_health_counseling_train.jsonl",
    "content": open("mental_health_counseling_train.jsonl", "rb"),
})
mental_health_counseling_eval = client.files.upload(file={
    "file_name": "mental_health_counseling_eval.jsonl",
    "content": open("mental_health_counseling_eval.jsonl", "rb"),
})

In [20]:
# This cell is optional. It retrieves the uploaded files
# Only needed if you don't execute the above cell again
files = client.files.list()
for file in files.data:
  if file.filename == "mental_health_counseling_train.jsonl":
    mental_health_counseling_train = file
  if file.filename == "mental_health_counseling_eval.jsonl":
    mental_health_counseling_eval = file

In [21]:
print(mental_health_counseling_train)

id='924c2193-5dc9-4e10-8cb7-6be32abc1ec3' object='file' bytes=4490329 created_at=1728660991 filename='mental_health_counseling_train.jsonl' sample_type='instruct' source='upload' PURPOSE='fine-tune' num_lines=3156


In [22]:
print(mental_health_counseling_eval)

id='f221d95f-d32b-4725-a056-52da3b80f0d4' object='file' bytes=496713 created_at=1728660992 filename='mental_health_counseling_eval.jsonl' sample_type='instruct' source='upload' PURPOSE='fine-tune' num_lines=352


## Create a fine-tuning job

In [141]:
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": mental_health_counseling_train.id, "weight": 1}],
    validation_files=[mental_health_counseling_eval.id],
    hyperparameters={
    "training_steps": 100,
    "learning_rate":0.0001
    },
    auto_start=True
)
created_jobs

JobOut(id='89cbe5c9-2517-47df-8963-da040dbc0b7b', auto_start=True, hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1728661718, modified_at=1728661719, training_files=['924c2193-5dc9-4e10-8cb7-6be32abc1ec3'], validation_files=['f221d95f-d32b-4725-a056-52da3b80f0d4'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))

In [142]:
print(created_jobs)

JobOut(id='89cbe5c9-2517-47df-8963-da040dbc0b7b', auto_start=True, hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1728661718, modified_at=1728661719, training_files=['924c2193-5dc9-4e10-8cb7-6be32abc1ec3'], validation_files=['f221d95f-d32b-4725-a056-52da3b80f0d4'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))


In [144]:
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
retrieved_jobs

DetailedJobOut(id='89cbe5c9-2517-47df-8963-da040dbc0b7b', auto_start=True, hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=11.675939931336865, fim_ratio=None), model='open-mistral-7b', status='RUNNING', job_type='FT', created_at=1728661718, modified_at=1728661722, training_files=['924c2193-5dc9-4e10-8cb7-6be32abc1ec3'], validation_files=['f221d95f-d32b-4725-a056-52da3b80f0d4'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=2100, cost=0.0, cost_currency=None, train_tokens_per_step=131072, train_tokens=13107200, data_tokens=1122582, estimated_start_time=None), events=[EventOut(name='status-updated', created_at=1728661722, data={'status': 'RUNNING'}), EventOut(name='status-updated', created_at=1728661722, data={'status': 'QUEUED'}), EventOut(name='status-updated', created_at=1728661722, data={'status': 'VAL

In [145]:
# Check the fine-tuning status every 60 seconds until completed
import time

retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
    print(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 60 seconds")
    time.sleep(60)

DetailedJobOut(id='89cbe5c9-2517-47df-8963-da040dbc0b7b', auto_start=True, hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=11.675939931336865, fim_ratio=None), model='open-mistral-7b', status='RUNNING', job_type='FT', created_at=1728661718, modified_at=1728661722, training_files=['924c2193-5dc9-4e10-8cb7-6be32abc1ec3'], validation_files=['f221d95f-d32b-4725-a056-52da3b80f0d4'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=2100, cost=0.0, cost_currency=None, train_tokens_per_step=131072, train_tokens=13107200, data_tokens=1122582, estimated_start_time=None), events=[EventOut(name='status-updated', created_at=1728661722, data={'status': 'RUNNING'}), EventOut(name='status-updated', created_at=1728661722, data={'status': 'QUEUED'}), EventOut(name='status-updated', created_at=1728661722, data={'status': 'VAL

In [23]:
# List jobs
jobs = client.fine_tuning.jobs.list()
print(jobs)

total=3 data=[JobOut(id='89cbe5c9-2517-47df-8963-da040dbc0b7b', auto_start=True, hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=11.675939931336865, fim_ratio=None), model='open-mistral-7b', status='SUCCESS', job_type='FT', created_at=1728661718, modified_at=1728662343, training_files=['924c2193-5dc9-4e10-8cb7-6be32abc1ec3'], validation_files=['f221d95f-d32b-4725-a056-52da3b80f0d4'], OBJECT='job', fine_tuned_model='ft:open-mistral-7b:e4a88ecf:20241011:89cbe5c9', suffix=None, integrations=[], trained_tokens=13107200, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=2100, cost=0.0, cost_currency=None, train_tokens_per_step=131072, train_tokens=13107200, data_tokens=1122582, estimated_start_time=None)), JobOut(id='dedfbafa-be67-4d45-afbc-c701f5ea06fe', auto_start=True, hyperparameters=TrainingParameters(training_steps=200, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=23.35

In [148]:
# Retrieve a jobs
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
print(retrieved_jobs)

DetailedJobOut(id='89cbe5c9-2517-47df-8963-da040dbc0b7b', auto_start=True, hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=11.675939931336865, fim_ratio=None), model='open-mistral-7b', status='SUCCESS', job_type='FT', created_at=1728661718, modified_at=1728662343, training_files=['924c2193-5dc9-4e10-8cb7-6be32abc1ec3'], validation_files=['f221d95f-d32b-4725-a056-52da3b80f0d4'], OBJECT='job', fine_tuned_model='ft:open-mistral-7b:e4a88ecf:20241011:89cbe5c9', suffix=None, integrations=[], trained_tokens=13107200, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=2100, cost=0.0, cost_currency=None, train_tokens_per_step=131072, train_tokens=13107200, data_tokens=1122582, estimated_start_time=None), events=[EventOut(name='status-updated', created_at=1728662343, data={'status': 'SUCCESS'}), EventOut(name='status-updated', created_at=1728661722, data={'status': 'RUNNING'}), EventOut(name='status-updat

## Use a fine-tuned model

After a successful fine-tuning job, the model can be used immediately.

In [149]:
retrieved_jobs.fine_tuned_model

'ft:open-mistral-7b:e4a88ecf:20241011:89cbe5c9'

In [27]:
def run_model(prompt, model):
    chat_response = client.chat.complete(
      model = model,
      messages = [{"role":'user', "content": prompt}]
    )

    return chat_response.choices[0].message.content

In [25]:
finetuned_model = 'ft:open-mistral-7b:e4a88ecf:20241011:89cbe5c9'
base_model = 'open-mistral-7b'

Let's compare the fine-tuned model with the Mistral 7B base model.

_Note: If you're on the free tier and run into rate limiting errors, add a `time.sleep(1)` in between LLM calls_

In [30]:
prompt = 'I have a lot of issues going on right now. First of all, I have a lot of trouble sleeping at times, while other nights I sleep too much and still feel quite tired. Im also noticing increased irritability and experiencing anxiety attacks that last for hours. Is there something wrong with me and if so what should I do?'

print(run_model(prompt, finetuned_model))
print("\n\n-------\n\n")
print(run_model(prompt, base_model))

I would be curious to know a little bit more about your situation. The first things I would ask would be, what are you eating and drinking? How much are you exercising? Are you isolating or spending time with friends or family? Is there any particular thing that tends to lead up to your anxiety attacks? Is there anything that helps you feel better when you are having an anxiety attack? I would also recommend that you track your mood and what seems to be associated with it on a daily basis. If you have difficulty getting started with this, consider checking out the Mood and Food app.


-------


I'm really sorry that you're going through this, but I'm unable to provide a diagnosis or treatment recommendations. It's important to speak with a healthcare professional about your symptoms as they may be related to various physical or mental health conditions. Some common causes of sleep disturbances, irritability, and anxiety include stress, anxiety disorders, depression, bipolar disorder, s

In [31]:
prompt = "I'm feeling anxious about a big presentation. What should I do?"

print(run_model(prompt, finetuned_model))
print("\n\n-------\n\n")
print(run_model(prompt, base_model))

First acknowledge that you are feeling anxious, then begin to normalize that feeling. Public speaking is the #1 fear of people. So, you are not alone. Then, begin to do some deep breathing exercises. Inhale for 4, 6, 8, 10 counts and exhale for 6-8-10 counts. This will begin to slow down your racing heart. Visualize yourself giving the presentation and focus on one person in the audience and begin to tell yourself that you can do this. Also, try to get good rest the nights before and the night of your presentation. You want to be well rested when you present. Lastly, work on your presentation. Practice it in front of the mirror, friends or family. The more you practice it, the more confidence you will have in presenting it. You got this!


-------


I'm sorry to hear that you're feeling anxious. Here are some strategies that might help:

1. Preparation: Make sure you're well-prepared. Understand your topic thoroughly, practice your presentation, and know your audience.

2. Visualizatio

In [35]:
prompt = "I'm feeling down lately. I have a lot of pressure at work, and feel like I can't keep up. What should I do?"

print(run_model(prompt, finetuned_model))
print("\n\n-------\n\n")
print(run_model(prompt, base_model))

There is a lot of pressure on all of us to perform at work and to keep up with everyone else. For a lot of people, their self-worth is defined by the job they do and the amount they earn. Workplace stress is a major contributor to mental health problems. One of the best things you can do is to recognise this and do something about it. Stress can be a major block to our ability to perform well at our work and unfortunately, the more we try to push through it (often with the help of caffeine and sugary treats) the worse it can become. I'm not saying that you need to take a day off work, but maybe looking at what you can do to manage your stress levels is important. This can be anything from talking to your work about your concerns, looking at your work/life balance, getting regular exercise, eating a balanced diet, reducing caffeine and sugar intake, getting enough sleep, and looking at your thought patterns to ensure you are looking at the situation rationally, not all things are in our

In [84]:
prompt = "What are some lifestyle changes that can be made to manage stress and anxiety?"

print(run_model(prompt, finetuned_model))
print("\n\n-------\n\n")
print(run_model(prompt, base_model))

There are many lifestyle changes that can help manage stress and anxiety. It's helpful to keep in mind that what is right for one person may not be the best for another. Some general suggestions are to pay more attention to your diet and exercise. Eating a healthy diet that includes lots of fruits and vegetables, lean proteins, and whole grains can help your body get the nutrients it needs to function properly. Additionally, decreasing consumption of sugar, caffeine, and alcohol can also help. Exercise is also important. I always tell my clients to try and fit in at least 30 minutes of exercise 5 days a week, but if you can only do 10 minutes a day, that's better than nothing. It's also important to find an exercise that you enjoy. Yoga, pilates, and tai chi are all great low-impact exercises that can also help with mindfulness. Another way to engage in mindfulness is by practicing meditation. There are many apps, such as Headspace and Calm, that can help you get started. Getting enoug

## Safeguarding with an LLM supervisor

Use a second LLM to rate the severity of the mental health problem. In case of a high severity, you can take extra care and further action.

Play around with this prompt, e.g. judging can be based on only the user question, or it's possible to assign different categories (e.g. Anxiety, Depression, Suicidal, Stress, Insomnia, Normal). Here, we look at both question and response and want to create a score of the severity.

In [75]:
# Also possible to categorize different states, e.g. Anxiety, Depression, Suicidal, Stress, Insomnia, Normal
scoring_prompt = """The following is a user question and an assistant response
from a mental health counseling conversation. Based on the provided question and response,
rate the severity of the user's mental health issues, ranging from 0 to 1. If you think the user
does not have a problem, assign the severity score 0.

Return a JSON object with severity as the key and the score as the value.

# question
{question}

# response
{response}
"""

In [68]:
def run_supervisor(question, response):
  prompt = scoring_prompt.format(
      question=question,
      response=response
  )

  messages = [{"role":"user", "content": prompt}]

  chat_response = client.chat.complete(
      model="mistral-large-latest",
      temperature=0,
      messages=messages,
      response_format={"type": "json_object"}
  )

  return chat_response.choices[0].message.content

In [76]:
prompt = "I'm feeling nervous about a big presentation. What should I do?"

response = run_model(prompt, finetuned_model)
print(run_supervisor(prompt, response))

{"severity": 0.1}


In [77]:
prompt = "I'm feeling down lately. I have a lot of pressure at work, and feel like I can't keep up. What should I do?"

response = run_model(prompt, finetuned_model)
print(run_supervisor(prompt, response))

{"severity": 0.4}


In [81]:
prompt = "I barely sleep anymore and feel so depressed. What can I do?"

response = run_model(prompt, finetuned_model)
print(run_supervisor(prompt, response))

{"severity": 0.8}


In [79]:
prompt = "I'm feeling good, what a nice day!"

response = run_model(prompt, finetuned_model)
print(run_supervisor(prompt, response))

{"severity": 0}


## Helpful resources

- Dataset: [Amod/mental_health_counseling_conversations](https://huggingface.co/datasets/Amod/mental_health_counseling_conversations)
- Mistral [Fine-tuning guide](https://docs.mistral.ai/guides/finetuning/)
- Guide on [LLM-based Evaluation](https://docs.mistral.ai/guides/evaluation/#llm-based-evaluation)
- If you need to talk to someone, see [https://findahelpline.com/](https://findahelpline.com/)